In [1]:
from src.jiggy.task import Node
from src.jiggy.pipeline import Pipeline

In [2]:
from copy import copy, deepcopy
from collections import deque

try:
    from collections import OrderedDict
except ImportError:
    print('got here')
    from ordereddict import OrderedDict


class DAGValidationError(Exception):
    pass
    
class DAG(object):
    """ Directed acyclic graph implementation. """
    
    library = dict()

    def __init__(self):
        """ Construct a new DAG with no nodes or edges. """
        self.reset_graph()
        
    def register_node(self, node):
        """Register node in library."""
        self.library.update({node.name: node})

    def add_node(self, node, graph=None):
        """ Add a node if it does not exist yet, or error out. """
        if not graph:
            graph = self.graph
        if node.name in graph:
            raise KeyError('node %s already exists' % node.name)
            
#         if self.library:
#             for idx, lm in enumerate(self.library):
#                 if any(x.get("dependency") ==lm.name for x in node.params):
                
#                     import pdb; pdb.set_trace()
#         self.library.append(node)
        graph[node.name] = set()

    def add_node_if_not_exists(self, node, graph=None):
        try:
            self.add_node(node, graph=graph)
        except KeyError:
            pass

#     def delete_node(self, node_name, graph=None):
#         """ Deletes this node and all edges referencing it. """
#         if not graph:
#             graph = self.graph
#         if node_name not in graph:
#             raise KeyError('node %s does not exist' % node_name)
#         graph.pop(node_name)

#         for node, edges in graph.items():
#             if node_name in edges:
#                 edges.remove(node_name)

#     def delete_node_if_exists(self, node_name, graph=None):
#         try:
#             self.delete_node(node_name, graph=graph)
#         except KeyError:
#             pass

    def add_edge(self, ind_node, dep_node, graph=None):
        """ Add an edge (dependency) between the specified nodes. """
        if not graph:
            graph = self.graph
        if ind_node not in graph or dep_node not in graph:
            raise KeyError('one or more nodes do not exist in graph')
        test_graph = deepcopy(graph)
        test_graph[ind_node].add(dep_node)
        is_valid, message = self.validate(test_graph)
        if is_valid:
            graph[ind_node].add(dep_node)
        else:
            raise DAGValidationError()

#     def delete_edge(self, ind_node, dep_node, graph=None):
#         """ Delete an edge from the graph. """
#         if not graph:
#             graph = self.graph
#         if dep_node not in graph.get(ind_node, []):
#             raise KeyError('this edge does not exist in graph')
#         graph[ind_node].remove(dep_node)

#     def rename_edges(self, old_task_name, new_task_name, graph=None):
#         """ Change references to a task in existing edges. """
#         if not graph:
#             graph = self.graph
#         for node, edges in graph.items():

#             if node == old_task_name:
#                 graph[new_task_name] = copy(edges)
#                 del graph[old_task_name]

#             else:
#                 if old_task_name in edges:
#                     edges.remove(old_task_name)
#                     edges.add(new_task_name)

    def predecessors(self, node, graph=None):
        """ Returns a list of all predecessors of the given node """
        if graph is None:
            graph = self.graph
        return [key for key in graph if node in graph[key]]

    def downstream(self, node, graph=None):
        """ Returns a list of all nodes this node has edges towards. """
        if graph is None:
            graph = self.graph
        if node not in graph:
            raise KeyError('node %s is not in graph' % node)
        return list(graph[node])

    def all_downstreams(self, node, graph=None):
        """Returns a list of all nodes ultimately downstream
        of the given node in the dependency graph, in
        topological order."""
        if graph is None:
            graph = self.graph
        nodes = [node]
        nodes_seen = set()
        i = 0
        while i < len(nodes):
            downstreams = self.downstream(nodes[i], graph)
            for downstream_node in downstreams:
                if downstream_node not in nodes_seen:
                    nodes_seen.add(downstream_node)
                    nodes.append(downstream_node)
            i += 1
        return list(
            filter(
                lambda node: node in nodes_seen,
                self.topological_sort(graph=graph)
            )
        )

    def all_leaves(self, graph=None):
        """ Return a list of all leaves (nodes with no downstreams) """
        if graph is None:
            graph = self.graph
        return [key for key in graph if not graph[key]]

    def from_dict(self, graph_dict):
        """ Reset the graph and build it from the passed dictionary.
        The dictionary takes the form of {node_name: [directed edges]}
        """

        self.reset_graph()
        for new_node in graph_dict.keys():
            self.add_node(new_node)
        for ind_node, dep_nodes in graph_dict.items():
            if not isinstance(dep_nodes, list):
                raise TypeError('dict values must be lists')
            for dep_node in dep_nodes:
                self.add_edge(ind_node, dep_node)

    def reset_graph(self):
        """ Restore the graph to an empty state. """
        self.graph = OrderedDict()

    def ind_nodes(self, graph=None):
        """ Returns a list of all nodes in the graph with no dependencies. """
        if graph is None:
            graph = self.graph

        dependent_nodes = set(
            node for dependents in graph.values() for node in dependents
        )
        return [node for node in graph.keys() if node not in dependent_nodes]

    def validate(self, graph=None):
        """ Returns (Boolean, message) of whether DAG is valid. """
        graph = graph if graph is not None else self.graph
        if len(self.ind_nodes(graph)) == 0:
            return (False, 'no independent nodes detected')
        try:
            self.topological_sort(graph)
        except ValueError:
            return (False, 'failed topological sort')
        return (True, 'valid')

    def topological_sort(self, graph=None):
        """ Returns a topological ordering of the DAG.
        Raises an error if this is not possible (graph is not valid).
        """
        if graph is None:
            graph = self.graph

        in_degree = {}
        for u in graph:
            in_degree[u] = 0

        for u in graph:
            for v in graph[u]:
                in_degree[v] += 1

        queue = deque()
        for u in in_degree:
            if in_degree[u] == 0:
                queue.appendleft(u)

        l = []
        while queue:
            u = queue.pop()
            l.append(u)
            for v in graph[u]:
                in_degree[v] -= 1
                if in_degree[v] == 0:
                    queue.appendleft(v)

        if len(l) == len(graph):
            return l
        else:
            raise ValueError('graph is not acyclic')

    def size(self):
        return len(self.graph)

In [3]:
pipe = Pipeline('examples/inputs/new_ext.yml')

In [4]:
pipe.tasks

[{'name': 'print-somethin',
  'description': 'Print something',
  'function': {'source': 'examples.utils.string_manipulation.PrintThis',
   'params': [{'type': 'str', 'value': 'THIS PIPELINE IS BONKERS'}],
   'output': None},
  'requires': None},
 {'name': 'print-somethin-2',
  'description': 'Print something',
  'function': {'source': 'examples.utils.string_manipulation.PrintThis',
   'params': [{'type': 'str', 'value': 'THIS PIPELINE IS BONKERS'}],
   'output': None},
  'requires': 'print-somethin-2'},
 {'name': 'print-somethin-3',
  'description': 'Print something',
  'function': {'source': 'examples.utils.string_manipulation.PrintThis',
   'params': [{'type': 'str', 'value': 'THIS PIPELINE IS BONKERS'}],
   'output': None},
  'requires': None},
 {'name': 'get-current-date',
  'description': 'Gets todays date',
  'function': {'source': 'examples.utils.dates.GetDateTask',
   'params': None,
   'output': [{'id': 'today-date', 'type': 'datetime'}]},
  'requires': 'print-somethin-2'},
 

In [5]:
dag = DAG()

In [6]:
for x in pipe.tasks:
    dag.register_node(Node(x))

In [7]:
dag.library

{'print-somethin': <Node `print-somethin`>,
 'print-somethin-2': <Node `print-somethin-2`>,
 'print-somethin-3': <Node `print-somethin-3`>,
 'get-current-date': <Node `get-current-date`>,
 'get-first-letter': <Node `get-first-letter`>,
 'get-weekday': <Node `get-weekday`>}

In [8]:
dag.all_leaves()

[]

In [9]:
dag.library

{'print-somethin': <Node `print-somethin`>,
 'print-somethin-2': <Node `print-somethin-2`>,
 'print-somethin-3': <Node `print-somethin-3`>,
 'get-current-date': <Node `get-current-date`>,
 'get-first-letter': <Node `get-first-letter`>,
 'get-weekday': <Node `get-weekday`>}

In [10]:
for task_name, task in dag.library.items():
    dag.add_node_if_not_exists(task)
#     print('Requires: ', task.requires)
#     print('Depends: ', task.params)
    upstreams = set()
    # if the task has parameters, register the dependency
    if task.params:
        for param in task.params:
            if param.get('dependency') in set(dag.library.keys()):
                upstreams.add(param.get('dependency'))
    if task.requires:
        upstreams.add(task.requires)
        
    if upstreams:
        for x in upstreams:
#             nd = [y for y in dag.library if y.name == x][0]  # need to solve
            dag.add_node_if_not_exists(dag.library.get(x))
            dag.add_edge(task.name, x)
        
    print(dag.graph)

OrderedDict([('print-somethin', set())])


DAGValidationError: 

In [ ]:
dag.topological_sort()[::-1]

In [ ]:
no_deps = []
dependents = []
for k, v in dag.graph.items():
    if v == set():
        no_deps.append((k, v))
    else:
        dependents.append((k,v))
no_deps.extend(dependents)
final = {x[0], x[1] for x}

In [ ]:
no_deps

In [ ]:
y = DAG()

In [ ]:
y.from_dict